In [1]:
import os
import re
import time
import random
import math
import requests
import json
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [2]:
# 配置浏览器
# chrome.exe --remote-debugging-port=9999 --user-data-dir="D:\Selenium\AutomationProfile"
# chrome.exe --remote-debugging-port=9999 --user-data-dir="D:\Program Files\ChromeDir"
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "localhost:9998")
driver = webdriver.Chrome(options=options)

driver.implicitly_wait(60)

The chromedriver version (123.0.6312.122) detected in PATH at D:\Program Files\ChromeDriver\chromedriver.exe might not be compatible with the detected chrome version (124.0.6367.61); currently, chromedriver 124.0.6367.60 is recommended for chrome 124.*, so it is advised to delete the driver in PATH and retry


In [3]:
# 读取信息
df_data= pd.read_excel(r"F:\collection_cgsi\collection_cgsi_info_gz.xlsx")

In [4]:
df_data.columns

Index(['OBJECTID', '钻孔编号', '钻孔类型', '钻孔深度', '实物档号', '项目名称', '保管单位', '数字化图形',
       '详情链接', 'dataid', 'lon', 'lat', '多参数扫描图', '钻孔结构化数据', '钻孔柱状图', '综合数据展示',
       'NAME', '省', '市', 'deep'],
      dtype='object')

In [5]:
search_data = df_data[['dataid','钻孔柱状图']]
search_data_copy =search_data.copy()

In [6]:
search_data_copy.dropna(inplace=True)

In [7]:
search_data_dict = dictionary_list = [{'key': row['dataid'], 'value': row['钻孔柱状图']} for _, row in search_data_copy.iterrows()]

In [10]:
for one_item in search_data_dict:
    if one_item['key'] == 'CE11020FDB9449409FDEA84B37B74565':
        print(one_item)
        print(search_data_dict.index(one_item))

{'key': 'CE11020FDB9449409FDEA84B37B74565', 'value': 'https://zkinfo.ndcp.cgsi.cn/histogram/drill/index?zkid=CE11020FDB9449409FDEA84B37B74565'}
1338


In [11]:
# 循环获取数据

data_info_list = []
error_info_list = []

for one_item in search_data_dict[1339:]:
    data_id = one_item['key']
    data_url = one_item['value']
    driver.get(data_url)
    print(data_id, data_url)
    try:
        # 等待用户输入数值
        deep_value = input(f'请输入{data_id}的深度值（输入9999退出）：')
        
        # 检查深度值是否为9999，如果是则退出循环
        
        if deep_value == '9999':
            print("\n您输入了9999，程序将退出。")
            break
        
        # 将数据添加到data_info_list中
        data_info_list.append({data_id: deep_value})
        
        # 将data_info_list追加写入文件
        with open(r"F:\collection_cgsi\point_info_result_zzt.json", 'a') as file:
            json.dump({data_id: deep_value}, file)
            file.write('\n')  # 每个数据项之间用换行符分隔

        
    except Exception as e:
        print(f'{data_id}的深度值获取失败: {e}')
        error_info_list.append(data_id)


# 循环结束后，如果需要，您还可以再次写入error_info_list或执行其他操作
with open(r"F:\collection_cgsi\point_info_result_zzt_error.json", 'w') as file:
    json.dump(error_info_list, file)
# 关闭浏览器
driver.quit()

0F74EC7EBCC6480A99B9531F58377E29 https://zkinfo.ndcp.cgsi.cn/histogram/drill/index?zkid=0F74EC7EBCC6480A99B9531F58377E29
568D21E0FCB748CC9B504671E3A47F07 https://zkinfo.ndcp.cgsi.cn/histogram/drill/index?zkid=568D21E0FCB748CC9B504671E3A47F07
5604B90B0ED74AED98456511D791E9C5 https://zkinfo.ndcp.cgsi.cn/histogram/drill/index?zkid=5604B90B0ED74AED98456511D791E9C5
2C7C006F7451454CA93C49A3D3771A4D https://zkinfo.ndcp.cgsi.cn/histogram/drill/index?zkid=2C7C006F7451454CA93C49A3D3771A4D
90A553743FB14DAE91CA70CD07E3278E https://zkinfo.ndcp.cgsi.cn/histogram/drill/index?zkid=90A553743FB14DAE91CA70CD07E3278E
39FD0B037CC347D0AFC3CC52F8B96362 https://zkinfo.ndcp.cgsi.cn/histogram/drill/index?zkid=39FD0B037CC347D0AFC3CC52F8B96362
B1EA9006667E4C3BB02659A716BB00FA https://zkinfo.ndcp.cgsi.cn/histogram/drill/index?zkid=B1EA9006667E4C3BB02659A716BB00FA
AD9FD337A4D94A4E925C0D6EC48024D7 https://zkinfo.ndcp.cgsi.cn/histogram/drill/index?zkid=AD9FD337A4D94A4E925C0D6EC48024D7
6CB047F2B1354C6A85A3B77FD0E7B700